# Project 2
## Trevis Slagowski & Treyson Grange

In [21]:
import pandas as pd 
import seaborn as sns
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [ ]:
zipcodes_df = pd.read_csv('datasets/AustinZipCodes.csv')
crime_df = pd.read_csv('datasets/crime-housing-austin-2015.csv')

## Data Exploration

In [ ]:
display(zipcodes_df.head())

In [ ]:
display(crime_df.head())

In [ ]:
crime_df.columns

In [ ]:
crime_df.describe()

In [ ]:
crime_df.info()

In [ ]:
display(crime_df['Location'].value_counts())

In [ ]:
display(crime_df['Clearance_Status'].value_counts())

## First Analysis

## Second Analysis

## Third Analysis

## Fourth Analysis